In [7]:
import argparse
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import os

from recbole.quick_start import load_data_and_model

## inference1

In [70]:
def inference_top_n(
    model_pth_path:str = '/opt/ml/recbole-test/RecBole/saved/BPR-Jun-07-2022_08-33-10.pth', 
    top_n:int = 10
    ) -> pd.DataFrame:
    
    _, model, dataset, _, _, test_data = load_data_and_model(model_pth_path)
    print('inference...')
    # device 설정
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # user, item id -> token 변환 array
    user_id2token = dataset.field2id_token['user_id']
    item_id2token = dataset.field2id_token['recipe_id']

    # user-item sparse matrix
    matrix = dataset.inter_matrix(form='csr')

    # user id, predict item id 저장 변수
    pred_list = None
    user_list = None

    model.eval()
    for data in tqdm(test_data):
        interaction = data[0].to(device)
        score = model.full_sort_predict(interaction)
        
        rating_pred = score.cpu().data.numpy().copy()
        batch_user_index = interaction['user_id'].cpu().numpy()
        rating_pred[matrix[batch_user_index].toarray()[0] > 0] = -np.inf
        ind = np.argpartition(rating_pred, -top_n)[ -top_n:]
        sorted_ind_by_pred = ind[np.argsort(rating_pred[ind])[::-1]]
        
        # 예측값 저장
        if pred_list is None:
            pred_list = sorted_ind_by_pred
            user_list = np.array([batch_user_index.item() for _ in range(top_n)])
        else:
            pred_list = np.append(pred_list, sorted_ind_by_pred)
            user_list = np.append(user_list, np.array([batch_user_index.item() for _ in range(top_n)]))
        
    result = []
    for user, pred in zip(user_list, pred_list):
        result.append((int(user_id2token[user]), int(item_id2token[pred])))
            
    # 데이터 저장
    result_df = pd.DataFrame(result, columns=["user", "item"])
    result_df.sort_values(by='user', inplace=True)
    result_df.reset_index(drop=True, inplace=True)
    print('inference done!')
    
    return result_df

In [71]:
test = inference_top_n('/opt/ml/recbole-test/RecBole/saved/LightGCN-Jun-07-2022_12-38-53.pth', 10)

All the same value in [rating] from [        user_id  recipe_id  date  rating
0             1          1     1     1.0
1             1          2     2     1.0
2             1          3     2     1.0
3             1          4     3     1.0
4             1          5     4     1.0
...         ...        ...   ...     ...
580516     4971      24948  6283     1.0
580517     8860     147302  6283     1.0
580518     9388      44071  6284     1.0
580519     6922     122587  6284     1.0
580520     9686     103347  6285     1.0

[580521 rows x 4 columns]_feat].


inference...


100%|██████████| 9723/9723 [00:32<00:00, 295.50it/s]


inference done!


In [98]:
test[test['user'] == 9722]

user   item
97220  9722  25898
97221  9722  34581
97222  9722  11419
97223  9722  14129
97224  9722   8785
97225  9722   7457
97226  9722  31674
97227  9722   9644
97228  9722  12025
97229  9722  26520

## inference2

In [ ]:
_, model, dataset, _, _, test_data = load_data_and_model('/opt/ml/recbole-test/RecBole/saved/LightGCN-Jun-07-2022_12-38-53.pth')

In [101]:
zip_lightgcn = (model, dataset, test_data)

In [103]:
model, dataset, test_data = zip_lightgcn

In [104]:
# device 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# user, item id -> token 변환 array
user_id2token = dataset.field2id_token['user_id']
item_id2token = dataset.field2id_token['recipe_id']

# user-item sparse matrix
matrix = dataset.inter_matrix(form='csr')

# user id, predict item id 저장 변수
pred_list = None
user_list = None

model.eval()

target = 1523


In [116]:
def make_user_preference(model, test_data, target):
    for data in test_data:
        interaction = data[0].to(device)
        batch_user_index = interaction['user_id'].cpu().numpy()
        if user_id2token[batch_user_index].item() == str(target):
            score = model.full_sort_predict(interaction)
            rating_pred = score.cpu().data.numpy().copy()
            rating_pred[matrix[batch_user_index].toarray()[0] > 0] = -np.inf
            return rating_pred[1:]

In [120]:
test = make_user_preference(model, test_data, 2352)

In [121]:
test

array([ 0.4546572 ,  0.90277934,  0.68665576, ...,  0.25117344,
        0.5840132 , -0.23490018], dtype=float32)

## etc

In [109]:
pd.read_csv('/opt/ml/recbole-test/RecBole/dataset/foodcom/foodcom.inter')['recipe_id:token'].nunique()

147302

In [21]:
a = np.load('/opt/ml/final-project-level3-recsys-13/modeling/foodcomImplicit/architects/user_factors.npy')
b = np.load('/opt/ml/final-project-level3-recsys-13/modeling/foodcomImplicit/architects/item_factors.npy')

In [22]:
tt = (a[0] @ b.T)

In [26]:
tt

array([ 0.00062362,  0.00018614,  0.00402131, ..., -0.00064635,
       -0.00069128, -0.00069128], dtype=float32)